In [1]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.svm import SVC

import os.path as op
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt

from utils import qcAggregate, unstratifiedSample
from models import AggregatedLearner

%matplotlib inline

### Step 1: Load (and replace) data

In [2]:
bp = '../data/'
exp = '../figures/'
bpp = op.join(bp, exp)
try:
    os.makedirs(bpp)
except FileExistsError:
    pass

In [3]:
df = pd.read_hdf('../data/aggregation_connect+feature+demo_dset100x1x1x20.h5')
df.drop(["edgecount", "globaleffic", "degree", "modularity",
         "assort", "avplength", "weight", "ccoeff", "betweenness",
         "plength", "age", "bmi", "cholesterol", "vo2max"], axis=1, inplace=True)
df.columns

Index(['subject', 'simulation', 'graph', 'sex'], dtype='object')

In [4]:
subs = list(df['subject'].unique())
for idx, row in df.iterrows():
    # Replace subids with unique numbers
    loc = subs.index(row.subject)
    row['subject'] = loc
    del loc
    
    row['sex'] = int(row['sex'])
    
    # Replace graphs with some ID signal but a lot of common signal
    np.random.seed(row.subject + row.sex+4)
    g = 10*(np.random.random(row.graph.shape) - 0.5) + int(row.sex)*0.73
    np.random.seed(422)
    g += 20 * (np.random.random(row.graph.shape) - 0.2)
    row['graph'] = g
    del g

In [5]:
# clf = StackingClassifier
pca = PCA(n_components=20)
svc = SVC(kernel='linear', probability=True)
pipe = Pipeline(steps=[('pca', pca), ('svc', svc)])

rs = 41  # Determined by: start at 42, decrement until holdout set was balanced across sex
clfagg = AggregatedLearner(df, pipe, target_id='sex', observation_id='simulation',
                           sample_id='subject', data_id='graph', random_seed=rs,
                           verbose=False)

In [8]:
clfagg.fit(aggregation='ref')
clfagg.fit(aggregation='median')
clfagg.fit(aggregation='consensus')
clfagg.fit(aggregation='mean')
clfagg.fit(aggregation='mega')
clfagg.fit(aggregation='none')
clfagg.fit(aggregation='meta')

In [9]:
clfagg.performance_report()

,aggregation,acc,f1,test_acc,test_f1,n_models
0,ref,0.931944,0.912698,0.800000,0.750000,10
1,median,0.931944,0.912698,0.800000,0.750000,10
2,consensus,0.920833,0.903810,0.900000,0.857143,10
3,mean,0.931944,0.912698,0.800000,0.750000,10
4,mega,0.931885,0.912698,0.800000,0.750000,10
5,none,0.926581,0.904811,0.810891,0.753654,1010
6,meta,0.919444,0.893651,0.800000,0.750000,10
